## Since Dataset Is quite large:
### We will convert images from dicom to png and resize them.
**We will be using joblib parallel and delayed functions to speed up the process**

In [ ]:
import os
import random
import cv2
import glob2
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from joblib import Parallel, delayed
from tqdm import tqdm_notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as immg
from pathlib import Path
import os
import gc
import cv2
from tqdm.notebook import tqdm
import zipfile
import io
from sklearn.decomposition import PCA
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
tr_img_dir = Path('../input/vinbigdata-chest-xray-abnormalities-detection/train')
ts_img_dir = Path('../input/vinbigdata-chest-xray-abnormalities-detection/test')

In [ ]:
df = pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/sample_submission.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
files = [str(x)+'.dicom' for x in df.image_id.tolist()]

## SAMPLE Image

In [ ]:
def get_szs(x):
    fl = pydicom.read_file(x)
    return fl.Rows,fl.Columns

In [ ]:
#szs = Parallel(n_jobs=-1,verbose=1)(delayed(get_szs)(str(ts_img_dir)+'/'+i) for i in random.sample(files,50))

In [ ]:
IMAGE_SIZE = (512,512)

### Thanks to https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way

For Explaining and giving function for correct method for conversion.

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    img = cv2.resize(data, IMAGE_SIZE)
    
    return img

## We will use joblib **parallel & delayed** to read and process dicom images in batches of 50. 

In [ ]:
x_tot,x2_tot = [],[]
batch = 50
dim0 = []
dim1 = []
for idx in tqdm(range(0,len(files),batch)):
    names = [str(ts_img_dir)+'/'+x for x in files[idx:idx+batch]]
    out = Parallel(n_jobs=-1)(delayed(get_szs)(i) for i in names)
    for s in range(len(out)):
        d0,d1 = out[s][0],out[s][1]
        dim0.append(d0)
        dim1.append(d1)

In [ ]:
df['dim0'] = dim0
df['dim1'] = dim1

In [ ]:
df.to_csv('vinbigdata_test_meta.csv',index=False)